In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing  import LabelEncoder
from sklearn.preprocessing  import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation,BatchNormalization,Dropout,Flatten
from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt
import tensorflow
from pathlib import Path
import os


In [ ]:
repo_path = os.getcwd()
repo_path = Path(repo_path).parent
path=Path("veri_seti\\veri_seti.csv") 
df=pd.read_csv(str(repo_path)+"\\"+str(path), sep=',', encoding = 'utf8')

In [ ]:
le = LabelEncoder()
df["Gece/Gunduz"]=le.fit_transform(df["Gece/Gunduz"])
df["Haftasonu/Haftaici"]=le.fit_transform(df["Haftasonu/Haftaici"])
df["Mevsim"]=le.fit_transform(df["Mevsim"])
labels=LabelEncoder().fit_transform(df["IstasyonTipi"])
y = labels
x = df.drop(["IstasyonTipi"], axis=1)
classes=list(le.fit(df["IstasyonTipi"]).classes_)
nb_classes= len(classes)
nb_features = x.shape[1]

In [ ]:
x = StandardScaler().fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.25) # 0.25 x 0.8 = 0.2
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)
y_test = to_categorical(y_test)
x_train  = np.array(x_train).reshape(x_train.shape[0], x_train.shape[1],1)
x_valid  = np.array(x_valid).reshape(x_valid.shape[0], x_valid.shape[1],1)
x_test  = np.array(x_test).reshape(x_test.shape[0], x_test.shape[1],1)

In [ ]:
model = Sequential()
model.add(LSTM(512,input_shape = (nb_features,1)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.15))
model.add(Dense(2048, activation = "relu"))
model.add(Dropout(0.15))
model.add(Dense(1024, activation = "relu"))
model.add(Dense(nb_classes, activation="softmax"))
model.summary()


from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])
score = model.fit(x_train, y_train, epochs = 50, validation_data=(x_valid,y_valid),batch_size=32)

In [ ]:
print(("Ortalama Eğitim Kaybı: ", np.mean(model.history.history["loss"])))
print(("Ortalama Eğitim Başarımı: ", np.mean(model.history.history["accuracy"])))
print(("Ortalama Doğrulama Kaybı: ", np.mean(model.history.history["val_loss"])))
print(("Ortalama Doğrulama Başarımı: ", np.mean(model.history.history["val_accuracy"])))
print(("Ortalama F1-Skor Değeri: ", np.mean(model.history.history["f1_m"])))
print(("Ortalama Kesinlik Değeri: ", np.mean(model.history.history["precision_m"])))
print(("Ortalama Duyarlılık Değeri: ", np.mean(model.history.history["recall_m"])))
print(("Ortalama F1-Skor Değeri: ", np.mean(model.history.history["val_f1_m"])))
print(("Ortalama Kesinlik Değeri: ", np.mean(model.history.history["val_precision_m"])))
print(("Ortalama Duyarlılık Değeri: ", np.mean(model.history.history["val_recall_m"])))

In [ ]:
plt.plot(model.history.history["accuracy"])
plt.plot(model.history.history["val_accuracy"])
plt.title("Model Başarımları")
plt.ylabel("Başarım")
plt.xlabel("Epok Sayısı")
plt.legend(["Eğitim","Doğrulama"], loc="upper left")
plt.show()

plt.plot(model.history.history["loss"], color="g")
plt.plot(model.history.history["val_loss"], color="r")
plt.title("Model Kayıpları")
plt.ylabel("Kayıp")
plt.xlabel("Epok Sayısı")
plt.legend(["Eğitim","Doğrulama"], loc="upper left")
plt.show()

plt.plot(model.history.history["f1_m"], color="y")
plt.plot(model.history.history["val_f1_m"], color="b")
plt.title("Model F1-Skor")
plt.ylabel("F1-Skor")
plt.xlabel("Epok Sayısı")
plt.legend(["Eğitim","Doğrulama"], loc="upper left")
plt.show()

plt.plot(model.history.history["recall_m"], color="y")
plt.plot(model.history.history["val_recall_m"], color="b")
plt.title("Model Duyarlılığı")
plt.ylabel("Duyarlılık")
plt.xlabel("Epok Sayısı")
plt.legend(["Eğitim","Doğrulama"], loc="upper left")
plt.show()

plt.plot(model.history.history["precision_m"], color="y")
plt.plot(model.history.history["val_precision_m"], color="b")
plt.title("Model Kesinliği")
plt.ylabel("Kesinlik")
plt.xlabel("Epok Sayısı")
plt.legend(["Eğitim","Doğrulama"], loc="upper left")
plt.show()

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=128)
print("Kayıp" , results[0])
print("Accuracy" , results[1])
print("F1" , results[2])
print("Kesinlik" , results[3])
print("Duyarlılık" , results[4])

isimler = ['Kayıp', 'Başarım','F1-Skor','Kesinlik','Duyarlılık']
degerler =  [ results[0],  results[1], results[2], results[3], results[4]]
plt.bar(isimler, degerler)
plt.suptitle('Test Çıktılarının Karşılaştırılması')
plt.show()

In [ ]:
grup = ['Eğitim', 'Doğrulama','Test']
degerler =  [x_train.shape[0], x_valid.shape[0], x_test.shape[0]]
plt.bar(grup, degerler)
plt.suptitle('Veri Gruplarının Karşılaştırılması')
plt.show()
